In [ ]:
!pip install envs
!pip install python-dotenv
!pip install psycopg2-binary

In [ ]:
import sys
sys.path.insert(0,'..')
from utils.Metadata import Metadata
from utils.Database import Database
from utils.outlier import isOutlier

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math 
plt.style.use('classic')

meta = Metadata()
db = Database()

dfOpt = meta.dfOptimos()

dfEjec = meta.dfEjecuciones(db.getEjecucionesResultados())
dfEjec = dfEjec.merge(dfOpt,how='left',on='instance').set_index(['id_ejec'])
tabla = dfEjec

# cambiamos algorithm a categorical
jerarquia = ['GWO_SCP_BCL1_CPU_C','GWO_SCP_QL1_CPU_C','ANTHHGWO_SCP_FERO_CPU_C','ANTHHGWO_SCP_QL1_CPU_C']
tabla["algorithm"] = pd.Categorical(tabla["algorithm"],jerarquia)

tabla['exec_time_min'] = (tabla.date_end-tabla.date_start).astype('timedelta64[m]')
tabla['exec_time_hour'] = tabla.exec_time_min/60

## Estado de ejecuciones

In [ ]:
dfEjec.reset_index().groupby(['algorithm','instance','status'])['id_ejec'].count().\
to_frame().\
reset_index().\
pivot(index=['algorithm','instance'],columns='status',values='id_ejec')

## como vamos con los resultados

In [ ]:
dfEjec[(dfEjec.status=='terminado')&(~dfEjec.fitness.isna())].reset_index().\
groupby(['algorithm','instance','status']).\
agg({'optimo':['min'],'fitness':['min','max'],'id_ejec':['count'],'exec_time_hour':['mean']})

<hr>

## Detección de outliers para reprocesar

In [72]:
print('Marcando outliers....')
for alg in tabla.algorithm.unique():
    print(f'Algoritmo: {alg}')
    for idx,row in tabla.query("algorithm==@alg").iterrows():
        condicion = (tabla.instance == row['instance'])&(tabla.algorithm==row['algorithm'])&(tabla.status=='terminado')
        serie = tabla[condicion]['fitness']
        data, outliers = isOutlier(serie)
        
        for i,r in tabla[condicion].iterrows():
            fitness = r['fitness']
            if fitness in outliers:
                tabla.loc[i,'outlier'] = 1
            else:
                tabla.loc[i,'outlier'] = 0
print('Ok')

Marcando outliers....
Algoritmo: ANTHHGWO_SCP_QL1_CPU_C
Algoritmo: GWO_SCP_BCL1_CPU_C
Algoritmo: ANTHHGWO_SCP_FERO_CPU_C
Algoritmo: GWO_SCP_QL1_CPU_C
Ok


## Resumen de resultados finales

In [73]:
resumen = tabla[tabla.status=='terminado'].reset_index().groupby(['algorithm','instance']).\
agg({'optimo':['min'],'fitness':['min','max','std'],'id_ejec':['count'],'exec_time_hour':['mean','std'],'outlier':['sum']})

resumen['reproceso'] = 31-resumen['id_ejec']['count']+resumen['outlier']['sum']
resumen['reproceso'] = resumen['reproceso'].fillna(0).astype(int).apply(lambda x:np.max([x,0]))
resumen['ejec_ok'] = resumen['id_ejec']['count']-resumen['outlier']['sum']

resumen.reset_index().to_csv(f'{directory}/resumen.csv',sep="|",decimal=",")

### Boxplots por instancia (sin outliers)

In [74]:
print('Generando boxplots por instancia - sin outliers -')
for instance in tabla.instance.unique():
    data = tabla.query("instance == @instance and outlier == 0")[['algorithm','fitness']]
    data.boxplot(figsize=(12,4),by=['algorithm'],column='fitness')
    plt.suptitle(f'Boxplot for instance {instance}')
    plt.title('')
    plt.savefig(f'{directory}/BoxplotSinOutlier_{alg}_{inst}.png',dpi=150)
    plt.savefig(f'{directory}/eps/BoxplotSinOutlier_{alg}_{inst}.eps',dpi=150)
    plt.close()
print('Boxplots ok')


print('Generando boxplot por instance - con outliers -')

for instance in tabla.instance.unique():
    data = tabla.query("instance == @instance")[['algorithm','fitness']]
    data.boxplot(figsize=(12,4),by=['algorithm'],column='fitness')
    plt.suptitle(f'Boxplot for instance {instance}')
    plt.title('')
    plt.savefig(f'{directory}/BoxplotConOutlier_{alg}_{inst}.png',dpi=150)
    plt.savefig(f'{directory}/eps/BoxplotConOutlier_{alg}_{inst}.eps',dpi=150)
    plt.close()
print('Boxplots ok')

Generando boxplots por instancia - sin outliers -
Boxplots ok
Generando boxplot por instance - con outliers -
Boxplots ok


### Plots de convergencia y exploración/explotación

In [2]:
iters = db.getIteracionesByAlgInst('ANTHHGWO_SCP_QL1_CPU_C','mscpd1')
dfIter = meta.dfIteraciones(iters)

In [4]:
dfIter.head(3)

,algorithm,cpu_time,ds,explor_dimensionality,explor_entropy,explor_hamming,explor_hussain,explor_inertia_momentum,explor_inertia_weigth,explor_leunggaoxu,fitness,id_ejec,iter,repairN
0,ANTHHGWO_SCP_QL1_CPU_C,0.451322,"S2,Standard",100.00,100.00,100.00,100.00,100.00,100.00,100.00,3114.0,28.0,0.0,0
1,ANTHHGWO_SCP_QL1_CPU_C,0.457518,"S2,Static",73.86,86.02,82.31,82.18,82.31,90.72,99.84,3114.0,28.0,1.0,0
2,ANTHHGWO_SCP_QL1_CPU_C,0.462093,"S2,Standard",0.00,0.00,98.60,0.00,0.00,98.28,99.24,3114.0,28.0,2.0,0


In [38]:
totalplots = len(tabla.drop_duplicates(['algorithm','instance']).index)
print(f'Iniciando generación de grupos de {totalplots} plots')
i=0
for alg in tabla.algorithm.unique():
    for inst in tabla[tabla.algorithm==alg].instance.unique():
        i+=1
        
        print(f'[{i}/{totalplots}] Consultando a la db')
        iters = db.getIteracionesByAlgInst(alg,instance)
        dfIter = meta.dfIteraciones(iters)
        
        print(f'[{i}/{totalplots}] Datos ok')
        
        
        dfIter.pivot(columns='id_ejec',values='fitness',index='iter').plot(figsize=(15,7),legend=False)
        plt.title(f'Convergence plot for algorithm {alg} solving instance {inst}')
        #plt.show()
        plt.savefig(f'{directory}Convergence_{alg}_{inst}.png',dpi=150)
        plt.savefig(f'{directory}/eps/Convergence_{alg}_{inst}.eps',dpi=150)
        plt.close()
        
        print(f'[{i}/{totalplots}] Convergence guardado ok')
        
        id_ejecs = tabla[(tabla.algorithm==alg)&(tabla.instance==inst)&(tabla.outlier==0)].reset_index()['id_ejec'].unique()
        
        for id_ejec in id_ejecs:
            
            for key in meta.getMetrics():
                
                try:
                
                    dfIter[dfIter.id_ejec == id_ejec][['iter',key]].set_index('iter').plot(figsize=(15,7),legend=False)
                    plt.title(f'Exploration plot for algorithm {alg} solving instance {inst} \n{meta.getMetrics()[key]} Metric')
                    plt.savefig(f'{directory}Exploration_{alg}_{inst}_{key}_{id_ejec}.png',dpi=150)
                    plt.savefig(f'{directory}eps/Exploration_{alg}_{inst}_{key}_{id_ejec}.eps',dpi=150)
                    plt.close()

                    print(f'[{i}/{totalplots}] Exploration-{key}, ejec: {id_ejec} guardado ok')
                except:
                    continue

Iniciando generación de grupos de 28 plots
[1/28] Consultando a la db
[1/28] Datos ok
[1/28] Convergence guardado ok
[1/28] Exploration-explor_hussain, ejec: 652 guardado ok
[1/28] Exploration-explor_inertia_weigth, ejec: 652 guardado ok
[1/28] Exploration-explor_dimensionality, ejec: 652 guardado ok
[1/28] Exploration-explor_leunggaoxu, ejec: 652 guardado ok
[1/28] Exploration-explor_entropy, ejec: 652 guardado ok
[1/28] Exploration-explor_hamming, ejec: 652 guardado ok
[1/28] Exploration-explor_inertia_momentum, ejec: 652 guardado ok
[1/28] Exploration-explor_hussain, ejec: 176 guardado ok
[1/28] Exploration-explor_inertia_weigth, ejec: 176 guardado ok
[1/28] Exploration-explor_dimensionality, ejec: 176 guardado ok
[1/28] Exploration-explor_leunggaoxu, ejec: 176 guardado ok
[1/28] Exploration-explor_entropy, ejec: 176 guardado ok
[1/28] Exploration-explor_hamming, ejec: 176 guardado ok
[1/28] Exploration-explor_inertia_momentum, ejec: 176 guardado ok
[1/28] Exploration-explor_hussain